<a href="https://colab.research.google.com/github/KohJJ820/MLIsFun/blob/main/MYR_Exchange_Rate_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Foreign exchange rate is the value of a country's currency in relation to another country's currency. In the forex market, you sell one currency and buy another. If the currency you buy increases against the currency you sell, you profit. The exchanging rate also serves as an indicator of the country's economic health where various economic factors can contribute to the rate of exchange. With these available factors available, we question is exchange rate predictable?



This project aims to analyze fluctuations in the Malaysian exchange rate and forecast the value of the Malaysian Ringgit (RM) over the next five(?) years based on Malaysian economic indicators. By leveraging economic factors like Gross Domestic Product (GDP), Employment Rate, and Bank Interest Rate from the official Government of Malaysia data, we seek to provide insights into the future performance of the RM using Machine Learning.

The goal is to harness the power of data-driven decision-making in exchanging currency. Empowering the consumers’ purchasing power to manage their budget to buy imported goods and services, as well as the investors in the Forex market to anticipate the return on exchanging foreign currencies.


# Previous Work

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8002544

# Exploratory Data Analysis (EDA)




According to Investopedia, it listed 5 enconomic factors that can be used to forecast exchange rate. Among them are Gross Domestic Product (GDP), Employment data and Bank Interest Rate.



The data we collected are from the respective sources as shown in Table below.

| Economic Indicator | Source                  |
|--------------------|------------------------------|
| Exchange Rate                | World Bank       |
| Real GDP                | Government of Malaysia       |
| Employment Rate    | Government of Malaysia|
| Bank Interest Rate      | Government of malaysia |




#### Data Preprocessing

Based on the economic indicators for our exchange rate forecast, the four datasets collected are as follows:
1. Annual Principal Labour Force Statistics (Employment Rate)
2. Annual Real Gross Domestic Product 
3. Annual Interest Rates
4. Official exchange rate (LCU per US$, period average)

The following .csv files represents each dataset of the indicators, obtained from their respective source:
| Dateset | .csv | Source URL
|---------|------|--------------
| 1  | [employment.csv](datasets/employment.csv) | https://data.gov.my/data-catalogue/lfs_year
| 2  | [gdp.csv](datasets/gdp.csv) | https://data.gov.my/data-catalogue/gdp_gni_annual_real
| 3  | [interest_rates_annual.csv](datasets/interest_rates_annual.csv) | https://data.gov.my/data-catalogue/interestrates_annual
| 4  | [exchangeRate.csv](datasets/exchangeRate.csv) | https://data.worldbank.org/indicator/PA.NUS.FCRF?locations=MY



##### Imports & Setup

For reading and processing .csv files, we first import the pandas library to read .csv files into DataFrames.

In [157]:
# If not already installed, do: pip install pandas fastparquet
import pandas as pd
import math


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


##### Dataset 1: Annual Principal Labour Force Statistics (Employment Rate)

The annual principal labour force statistics includes the following data used:
| Data | Variable | Definition 
|------|----------|------------
| date (Date) | Date | The date in YYYY format
| u_rate (Float) | Unemployment Rate | Ratio of unemployed to labour force size
| ep_ratio (Float) | Employment-Population Ratio | Ratio of the number of employed people to the working-age (15-64) population

In [158]:
# Read employment data from 'employment.csv'
df_EMP = pd.read_csv('datasets/employment.csv', usecols=['date', 'ep_ratio', 'u_rate'])

# convert 'date' column to datetime format (in year)
df_EMP['date'] = pd.to_datetime(df_EMP['date']).dt.year

print(df_EMP)

    date  ep_ratio  u_rate
0   1982      62.7     3.4
1   1983      63.2     3.8
2   1984      62.0     5.0
3   1985      62.0     5.6
4   1986      61.2     7.4
5   1987      61.7     7.3
6   1988      62.0     7.2
7   1989      62.4     5.7
8   1990      63.6     4.5
9   1992      63.5     3.7
10  1993      63.8     4.1
11  1995      62.7     3.1
12  1996      64.6     2.5
13  1997      64.0     2.4
14  1998      62.2     3.2
15  1999      62.0     3.4
16  2000      63.4     3.0
17  2001      62.6     3.5
18  2002      62.1     3.5
19  2003      62.9     3.6
20  2004      62.1     3.5
21  2005      61.0     3.5
22  2006      61.0     3.3
23  2007      61.2     3.2
24  2008      60.6     3.3
25  2009      60.6     3.7
26  2010      61.6     3.3
27  2011      62.5     3.1
28  2012      63.6     3.0
29  2013      65.2     3.1
30  2014      65.7     2.9
31  2015      65.8     3.1
32  2016      65.4     3.4
33  2017      65.7     3.4
34  2018      66.0     3.3
35  2019      66.4     3.3
3

##### Dataset 2: Annual Real Gross Domestic Product

Long time series of annual real gross domestic product (GDP), including per capita values. 

The collected and processed data are as follows: 
| Data | Variable | Definition 
|------|----------|------------
| date (Date) | Date | The date in YYYY format
| gdp (Float, RM mil) | GDP | The total value of goods and services produced within that year, after deducting the cost of goods and services used in production, but before deducting the consumption of fixed capital. The values are in constant 2015 prices, i.e. with base 2015 = 100.
| gdp_capita (Float, RM) | GDP per Capita | The ratio of GDP to the total population of Malaysia in that year

In [159]:
# Read Real GPD data from 'gdp.csv'
df_GDP = pd.read_csv('datasets/gdp.csv', usecols=['date', 'series', 'gdp', 'gdp_capita'])

# filtering dataframe
df_GDP = df_GDP[df_GDP['series'] == 'abs'] 
df_GDP.drop(columns=['series'], inplace=True)

df_GDP['date'] = pd.to_datetime(df_GDP['date']).dt.year

print(df_GDP)

    date          gdp  gdp_capita
0   1970    73709.682    6773.390
1   1971    81106.199    7265.660
2   1972    88720.810    7751.699
3   1973    99102.104    8455.832
4   1974   107346.073    8941.871
5   1975   108205.988    8798.552
6   1976   120718.377    9589.923
7   1977   130077.743   10082.193
8   1978   138732.976   10509.519
9   1979   151703.260   11223.580
10  1980   162992.754   11743.564
11  1981   174307.881   12226.928
12  1982   184667.264   12604.099
13  1983   196213.087   13039.860
14  1984   211442.735   13684.346
15  1985   209274.918   13175.348
16  1986   211871.172   12973.946
17  1987   222871.380   13286.801
18  1988   245019.713   14229.576
19  1989   267217.521   15128.372
20  1990   291289.883   16090.387
21  1991   319094.864   17203.576
22  1992   347446.820   18223.312
23  1993   381826.485   19479.429
24  1994   417000.501   20702.562
25  1995   457987.835   22145.584
26  1996   503798.988   23737.032
27  1997   540690.887   24836.478
28  1998   500

##### Dataset 3: Annual Interest Rates

Various interest rates monitored by Bank Negara Malaysia, including Fixed Deposit Rate (FDR), Base Lending Rate (BLR), and Avg Lending Rate (ALR).

Our dataset collection focuses on the following data, specifically on commercial banks:
| Data | Variable | Definition 
|------|----------|------------
| date (Date) | Date | The date in YYYY format
| value of rate type (Float) | Interest Rate | Average rate of interest expressed as a percentage (%). The __type of interest rate = ['fdr_12mo', 'blr', 'alr']__

In [160]:
# Read bank interest rate data from 'interest_rates_annual.csv'
df_INS = pd.read_csv('datasets/interest_rates_annual.csv')

# Filter and extract dataframes for each interest rate type of commercial banks
df_INS = df_INS[df_INS['bank'] == 'commercial']
df_INS_dict = {
    'fdr_12mo': None,
    'blr': None,
    'alr': None,
}
for rate in df_INS_dict:
  df = df_INS.copy()
  df = df[df['rate'] == rate]
  df.rename(columns={'value': rate}, inplace=True)
  df.drop(columns=['bank', 'rate'], inplace=True)
  df_INS_dict[rate] = df

# Merge each dataframes of different interest rate type to a single dataframe
df_INS = df_INS_dict['fdr_12mo'].merge(df_INS_dict['blr'], how='left', on='date')
df_INS = df_INS.merge(df_INS_dict['alr'], how='left', on='date')

df_INS['date'] = pd.to_datetime(df_INS['date']).dt.year


print(df_INS)

    date   fdr_12mo        blr        alr
0   1980   7.500000   7.750000  10.130000
1   1981   9.937500   8.500000  11.890000
2   1982  10.480000   8.500000  12.395000
3   1983   8.750000  11.020833  11.775000
4   1984   9.615000  11.354167  12.210000
5   1985   9.291667  11.541667  12.552500
6   1986   7.416667  10.687500  12.230000
7   1987   4.500000   8.187500  10.362500
8   1988   4.240000   7.250000   9.300000
9   1989   5.202500   6.995833   8.895833
10  1990   6.214167   7.156667   8.785833
11  1991   7.483333   8.024167   9.347500
12  1992   8.031667   9.215000  10.161667
13  1993   6.954167   8.852500  10.030833
14  1994   5.463333   7.290833   8.764167
15  1995   6.502500   7.440833   8.730000
16  1996   7.180000   8.892500   9.943333
17  1997   7.827500   9.531667  10.627500
18  1998   8.563333  10.612500  12.156667
19  1999   4.286667   7.285833   8.561667
20  2000   0.000000   6.771667   7.674167
21  2001   4.000000   6.657500   7.125000
22  2002   4.000000   6.390000   6

##### Dataset 4: Official exchange rate (LCU per US$, period average)

The exchange rate determined by World Bank as an annual average based on monthly averages (local currency units relative to the U.S. dollar).

Only the exchange rate of Malaysia is processed:
| Data | Variable | Definition 
|------|----------|------------
| date (Date) | Date | The date in YYYY format
| exchange_rate (Float) | Exchange Rate | The exchange rate of myr_usd

In [161]:
# Read exchange rate data from 'exchangeRate.csv'
df_EX_ver = pd.read_csv('datasets/exchangeRate.csv')

# Extract and filter data for Malaysia country
df_EX_ver = df_EX_ver[df_EX_ver['Country Code'] == 'MYS']
df_EX_ver.drop(columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], inplace=True)


# Create a new dataframe with desired column-row format and append each data into it
df_EX = pd.DataFrame({'date': [], 'exchange_rate': []})

i = 0   # starting index for new dataframe
j = df_EX_ver.first_valid_index()  # get index of the row to retrieve value from old dataframe
for col in df_EX_ver.columns:
    val = df_EX_ver.at[j, col]

    if not math.isnan(val):
        df_EX.loc[i] = [col, val]
        i += 1

df_EX['date'] = pd.to_datetime(df_EX['date']).dt.year

print(df_EX)

    date  exchange_rate
0   1960       3.061220
1   1961       3.061220
2   1962       3.061220
3   1963       3.061220
4   1964       3.061220
5   1965       3.061220
6   1966       3.061220
7   1967       3.061220
8   1968       3.061220
9   1969       3.061220
10  1970       3.061220
11  1971       3.052260
12  1972       2.819556
13  1973       2.443330
14  1974       2.407067
15  1975       2.393783
16  1976       2.541575
17  1977       2.461283
18  1978       2.316042
19  1979       2.188442
20  1980       2.176883
21  1981       2.304125
22  1982       2.335392
23  1983       2.321250
24  1984       2.343642
25  1985       2.483042
26  1986       2.581442
27  1987       2.519638
28  1988       2.618783
29  1989       2.708842
30  1990       2.704875
31  1991       2.750067
32  1992       2.547383
33  1993       2.574095
34  1994       2.624257
35  1995       2.504404
36  1996       2.515942
37  1997       2.813192
38  1998       3.924375
39  1999       3.800000
40  2000       3

##### Dateset Merge

Once all datasets have been processed, the DataFrames of each dataset is merged together and exported as [datasets.csv](datasets/dataset.csv), which consists of following attributes:

| year | ep_ratio | u_rate | gdp | gdp_capita | fdr_12mo | blr | alr | exchange_rate
|------|----------|--------|-----|------------|----------|-----|----|---------------
| Year of the data | Employment-Population Ratio |  Unemployment Rate | Gross Domestic Product (GPD) | GDP per Capita | Fixed Demosit Rate (FDR) | Base Lending Rate (BLR) | Average Lending Rate (ALR) | Exchange Rate (myr_usd)

The dataset records the data of each indicators from the year 1982 to 2022.

In [162]:
# merge all DataFrames into a single dataset

df = df_EMP.merge(df_GDP, how='left', on='date')
df = df.merge(df_INS, how='left', on='date')
df = df.merge(df_EX, how='left', on='date')

# rename 'data' column to 'year' for more concise naming and set it as the index
df.rename(columns={'date': 'year'}, inplace=True)
df.set_index('year', inplace=True)

# export the merged dataset as csv
df.to_csv('datasets/dataset.csv')
print(df)

      ep_ratio  u_rate          gdp  gdp_capita   fdr_12mo        blr  \
year                                                                    
1982      62.7     3.4   184667.264   12604.099  10.480000   8.500000   
1983      63.2     3.8   196213.087   13039.860   8.750000  11.020833   
1984      62.0     5.0   211442.735   13684.346   9.615000  11.354167   
1985      62.0     5.6   209274.918   13175.348   9.291667  11.541667   
1986      61.2     7.4   211871.172   12973.946   7.416667  10.687500   
1987      61.7     7.3   222871.380   13286.801   4.500000   8.187500   
1988      62.0     7.2   245019.713   14229.576   4.240000   7.250000   
1989      62.4     5.7   267217.521   15128.372   5.202500   6.995833   
1990      63.6     4.5   291289.883   16090.387   6.214167   7.156667   
1992      63.5     3.7   347446.820   18223.312   8.031667   9.215000   
1993      63.8     4.1   381826.485   19479.429   6.954167   8.852500   
1995      62.7     3.1   457987.835   22145.584   6

# Machine Learning Techniques

#Result

# Conclusion